In [140]:
import autograd.numpy as np
from autograd import grad, hessian
import sys

In [141]:
def subproblem_solver(g, B, delta):
    """
    Solve the subproblem min{g^T s + 1/2 s^T B s : ||s|| <= delta}
    """
    # solve the unconstrained problem
    # check if B is singular
    if np.linalg.cond(B) > 1/sys.float_info.epsilon:
        s = -np.linalg.solve(B + 1e-6*np.eye(B.shape[0]), g)
    else:
        s = -np.linalg.solve(B, g)
    # check if the unconstrained solution is within the trust region
    if np.linalg.norm(s) <= delta:
        return s
    # solve the constrained problem
    s = -delta * g / np.linalg.norm(g)
    return s

In [142]:
def sr1_update(B, s, y):
    """
    Update the Hessian approximation B using the SR1 formula
    """
    Bs = np.dot(B, s)
    sy = np.dot(s, y)
    if sy > 1e-8:
        B = B + np.outer(y - Bs, y - Bs) / sy
    return B

In [143]:
def newton_with_sr1(x0, f, g, B, delta, tol=1e-6, max_iter=100):
    """
    Newton method with SR1 update
    """
    x = x0
    for i in range(max_iter):
        # solve the subproblem
        s = subproblem_solver(g(x), B, delta)
        # update x
        x = x + s
        # update B
        B = sr1_update(B, s, g(x) - g(x - s))
        # check the stopping criterion
        if np.linalg.norm(s) < tol:
            break
    return x

In [144]:
f = lambda x: 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
g = grad(f)
B = hessian(f)
x0 = np.array([0., 0.])
delta = 1
x = newton_with_sr1(x0, f, g, B, delta)
print(x)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional